## 0. Setup & Configuration
Responsible for imports, paths, session auth, and environment loading.

In [ ]:
# Install deps if needed (uncomment for first run)
# %pip install -r requirements.txt
import os
import re
from pathlib import Path
import pandas as pd
import requests
from tqdm import tqdm
from rapidfuzz import fuzz
import matplotlib.pyplot as plt
import unicodedata, string
from urllib.parse import quote
from dotenv import load_dotenv

DATA_PATH = Path("Company.csv")
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

# Secrets
load_dotenv()
COMPANIES_HOUSE_API_KEY = os.getenv("COMPANIES_HOUSE_API_KEY", "")
BASE_URL = "https://api.company-information.service.gov.uk"

# Session
session = requests.Session()
if COMPANIES_HOUSE_API_KEY:
    session.auth = (COMPANIES_HOUSE_API_KEY, "")

print("API key present:", bool(COMPANIES_HOUSE_API_KEY))
print("Data path exists:", DATA_PATH.exists())

## 1. Data Collection
Load the provided `Company.csv` exactly as-is.

In [ ]:
# Load input CSV
assert DATA_PATH.exists(), "Company.csv"
df_raw = pd.read_csv(DATA_PATH)
print("Rows:", len(df_raw))
display(df_raw.head())

## 2. Profiling
Lightweight schema/quality profile to understand columns, nulls, and uniqueness. These checks are **non-destructive**.

In [ ]:
# Simple schema/quality profile
def profile_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for col in df.columns:
        s = df[col]
        rows.append({
            "column": col,
            "dtype": s.dtype.name,
            "nulls": int(s.isna().sum()),
            "non_null": int(s.notna().sum()),
            "unique": int(s.nunique(dropna=True)),
            "sample_values": list(s.dropna().astype(str).head(3))
        })
    return pd.DataFrame(rows)

profile = profile_dataframe(df_raw)
display(profile)

null_pct = df_raw.isna().mean().sort_values(ascending=False)
display(null_pct.to_frame("null_pct"))


## 3. Cleansing (Non-destructive normalization)
Create derived, normalized fields while preserving originals. Goals:

- Handle whitespace, casing, punctuation, and accents in names
- Normalize UK postcodes to a consistent format
- Normalize emails to lowercase
- Build a `block_key` for within-block fuzzy deduplication

In [ ]:
import re, unicodedata, string

COMPANY_SUFFIXES = {
    "LTD": "LIMITED", "LTD.": "LIMITED",
    "LIMITED": "LIMITED", "PLC": "PLC",
    "LLP": "LLP", "L.L.P": "LLP"
}

def normalize_whitespace(s):
    if not isinstance(s, str):
        return ""
    s = s.replace("\n", " ").replace("\r", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def strip_punct(s):
    return "".join(ch for ch in s if ch not in set(string.punctuation))

def normalize_company_name(s):
    s = normalize_whitespace(s).upper()
    # remove accents
    s = unicodedata.normalize("NFKD", s)
    s = "".join([c for c in s if not unicodedata.combining(c)])
    # Normalize common suffixes
    toks = s.split(" ")
    toks = [COMPANY_SUFFIXES.get(t, t) for t in toks]
    s = " ".join(toks)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalize_postcode(s):
    s = normalize_whitespace(str(s))
    s = s.upper().replace(" ", "")
    if len(s) > 3:
        s = s[:-3] + " " + s[-3:]
    return s.strip()

def normalize_email(s):
    s = normalize_whitespace(str(s)).lower()
    return s

def cleanse(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # company name
    name_col = next((c for c in out.columns if "company" in c.lower() and "name" in c.lower()), None)
    if name_col is None:
        out["company_name_norm"] = ""
    else:
        out["company_name_norm"] = out[name_col].astype(str).apply(normalize_company_name)

    # postcode
    postcode_col = next((c for c in out.columns if "postcode" in c.lower()), None)
    if postcode_col is None:
        out["postcode_norm"] = ""
    else:
        out["postcode_norm"] = out[postcode_col].astype(str).apply(normalize_postcode)

    # email (optional)
    email_col = next((c for c in out.columns if "email" in c.lower()), None)
    if email_col is None:
        out["email_norm"] = ""
    else:
        out["email_norm"] = out[email_col].astype(str).apply(normalize_email)

    # block key
    out["block_key"] = out["company_name_norm"].str.replace(" ", "", regex=False) + "|" + out["postcode_norm"].fillna("")
    return out

df_clean = cleanse(df_raw)
display(df_clean.head())

## 4. Deduplication
Two steps:

1. **Exact**: drop duplicates on normalized columns  
2. **Fuzzy within block**: cluster near-duplicates using `token_sort_ratio` and take a canonical record per cluster

In [ ]:
# Exact dedup on normalized keys
dedup_cols = ["company_name_norm","postcode_norm","email_norm"]
dedup_cols = [c for c in dedup_cols if c in df_clean.columns]
before = len(df_clean)
df_dedup = df_clean.drop_duplicates(subset=dedup_cols, keep="first").copy()
print(f"Exact dedup removed {before - len(df_dedup)} rows")

# Fuzzy clustering within blocks
def fuzzy_cluster_block(block_df: pd.DataFrame, name_col="company_name_norm", threshold=95):
    taken = set()
    clusters = []
    names = block_df[name_col].tolist()
    idxs = block_df.index.tolist()
    for i, idx in enumerate(idxs):
        if idx in taken:
            continue
        base = names[i]
        cl = [idx]
        taken.add(idx)
        for j in range(i+1, len(idxs)):
            jdx = idxs[j]
            if jdx in taken:
                continue
            score = fuzz.token_sort_ratio(base, names[j])
            if score >= threshold:
                cl.append(jdx)
                taken.add(jdx)
        clusters.append(cl)
    return clusters

clusters = []
for key, block_df in df_dedup.groupby("block_key"):
    if len(block_df) == 1:
        clusters.append([block_df.index[0]])
    else:
        clusters.extend(fuzzy_cluster_block(block_df, threshold=93))

canonical_idx = [c[0] for c in clusters]
df_canonical = df_dedup.loc[canonical_idx].copy().reset_index(drop=True)
print("Canonical records:", len(df_canonical), "from", len(df_dedup))

# optional mapping
cluster_map = {}
for cid, cl in enumerate(clusters):
    for idx in cl:
        cluster_map[idx] = cid
df_dedup = df_dedup.copy()
df_dedup["cluster_id"] = df_dedup.index.map(cluster_map)
df_dedup.to_csv(OUTPUT_DIR / "dedup_clusters.csv", index=False)

### Performance & rate limits
API calls dominate runtime. Expect variable latency and occasional 429s. The small retry/backoff in code keeps the run going without manual intervention.

## 5. Matching (Companies House) & Enrichment
**Rationale**: Local text similarity recovers probable matches. We then call Companies House to retrieve authoritative IDs and status.

In [ ]:
# -----------------------------------
# Companies House search function
# -----------------------------------
def ch_search_company(query: str, items_per_page: int = 5, retries: int = 2, backoff: float = 1.5):
    if not COMPANIES_HOUSE_API_KEY:
        raise RuntimeError("Set COMPANIES_HOUSE_API_KEY in .env before calling Companies House API.")

    url = f"{BASE_URL}/search/companies?q={quote(query)}&items_per_page={items_per_page}"

    for attempt in range(retries + 1):
        try:
            r = session.get(url, timeout=20)
        except Exception as e:
            print(f" Request failed for '{query}': {e}")
            return {"error": True, "status_code": None}

        # Handle rate limits with retry backoff
        if r.status_code == 429 and attempt < retries:
            time.sleep(backoff * (attempt + 1))
            continue

        # Status code error → mark as failed
        if r.status_code >= 400:
            return {"error": True, "status_code": r.status_code}

        try:
            return r.json()
        except Exception:
            return {"error": True, "status_code": r.status_code}

    return {"error": True, "status_code": 429}


# -----------------------------------
#  fuzzy match selector
# -----------------------------------
def best_match_for_name(name: str):
    data = ch_search_company(name, items_per_page=10)

    # FIX 1: Skip if API returned error or malformed response
    if not isinstance(data, dict) or data.get("error"):
        return None

    items = data.get("items", [])
    if not items:
        return None

    # Compute fuzzy match scores for all candidates
    candidates = []
    for it in items:
        reg_name = it.get("title") or it.get("company_name") or ""
        score = fuzz.token_set_ratio(name.upper(), reg_name.upper())
        candidates.append((score, it))

    if not candidates:
        return None

    # Sort by descending match score
    candidates.sort(reverse=True, key=lambda x: x[0])
    best_score, best_item = candidates[0]

    #  FIX 2: Safely flatten keys to avoid missing-field errors
    out = {
        "ch_best_title": best_item.get("title") or best_item.get("company_name"),
        "ch_best_company_number": best_item.get("company_number", None),
        "ch_best_company_status": best_item.get("company_status", None),
        "ch_best_address_snippet": best_item.get("address_snippet", None),
        "ch_best_date_of_creation": best_item.get("date_of_creation", None),
        "ch_best_score": best_score,
    }
    return out


# -----------------------------------
# Enrichment pipeline
# -----------------------------------
enrich_cols = [
    "ch_best_title",
    "ch_best_company_number",
    "ch_best_company_status",
    "ch_best_address_snippet",
    "ch_best_date_of_creation",
    "ch_best_score",
]

matches = []
for name in tqdm(df_canonical["company_name_norm"], desc="Matching Companies House"):
    try:
        m = best_match_for_name(name)
    except Exception as e:
        print(f"⚠️ Error for '{name}': {e}")
        m = None
    matches.append(m)

# FIX 3: Handle missing keys when normalizing
if any(matches):
    df_matches = pd.json_normalize(matches)
    for col in enrich_cols:
        if col not in df_matches.columns:
            df_matches[col] = None
    df_matches = df_matches[enrich_cols]
else:
    df_matches = pd.DataFrame(columns=enrich_cols)

# Merge canonical + enriched
df_enriched = pd.concat(
    [df_canonical.reset_index(drop=True), df_matches.reset_index(drop=True)],
    axis=1
)

display(df_enriched.head())

# Save to output folder
df_enriched.to_csv(OUTPUT_DIR / "enriched_companies.csv", index=False)
print(f"\n Enriched dataset saved to: {OUTPUT_DIR / 'enriched_companies.csv'}")

## 6. Reporting & Visualisation
KPIs and diagnostic charts.

### What to read in the results
- Match coverage and enriched count show how much the pipeline validated.
- Score histogram and status distribution help spot anomalies and input drift.
- Confidence bands (high/mid/low) guide what to accept vs send to review.


In [ ]:
# KPIs / coverage
kpis = {
    "input_rows": int(len(df_raw)),
    "after_exact_dedup": int(len(df_dedup)),
    "canonical_records": int(len(df_canonical)),
}

if "ch_best_company_number" in df_enriched.columns:
    kpis["enriched_records"] = int(df_enriched["ch_best_company_number"].notna().sum())
    kpis["match_rate_pct"] = round(100.0 * df_enriched["ch_best_company_number"].notna().mean(), 2)
else:
    kpis["enriched_records"] = 0
    kpis["match_rate_pct"] = 0.0

display(pd.DataFrame([kpis]))

# Status distribution
if "ch_best_company_status" in df_enriched.columns:
    status_counts = df_enriched["ch_best_company_status"].fillna("unmatched").value_counts().sort_values(ascending=False)
    plt.figure()
    status_counts.plot(kind="bar", title="Company status distribution")
    plt.xlabel("status"); plt.ylabel("count")
    plt.tight_layout()
    plt.show()

# Score histogram
if "ch_best_score" in df_enriched.columns:
    plt.figure()
    df_enriched["ch_best_score"].dropna().astype(float).plot(kind="hist", bins=20, title="Match score histogram")
    plt.xlabel("RapidFuzz token_sort_ratio"); plt.ylabel("freq")
    plt.tight_layout()
    plt.show()


### Evaluation (Proxy Metrics)
These metrics do not require labeled ground truth but still provide a quality signal.

In [ ]:
# These metrics do not require ground truth and do not change your logic.

eval_rows = []

if "ch_best_score" in df_enriched.columns:
    scores = df_enriched["ch_best_score"].fillna(0).astype(float)
    eval_rows.append({"metric":"avg_score", "value": float(scores.mean())})
    eval_rows.append({"metric":"p90_score", "value": float(scores.quantile(0.9))})
    eval_rows.append({"metric":"high_confidence_rate_95+", "value": float((scores>=95).mean())})
    eval_rows.append({"metric":"mid_confidence_rate_85_95", "value": float(((scores>=85)&(scores<95)).mean())})
    eval_rows.append({"metric":"low_confidence_rate_<85", "value": float((scores<85).mean())})

# Ambiguity proxy: name mismatch length for high scores
if set(["company_name_norm","ch_best_title","ch_best_score"]).issubset(df_enriched.columns):
    def token_set(s):
        return set(str(s).upper().split())
    def jaccard(a,b):
        a,b=token_set(a),token_set(b)
        if not a and not b: return 1.0
        return len(a&b)/max(1,len(a|b))
    mask = df_enriched["ch_best_score"].fillna(0).astype(float)>=90
    jacc = df_enriched.loc[mask].apply(lambda r: jaccard(r["company_name_norm"], r["ch_best_title"]), axis=1)
    eval_rows.append({"metric":"high_score_avg_name_jaccard", "value": float(jacc.mean())})

eval_df = pd.DataFrame(eval_rows)
display(eval_df)

## 7. Outputs
Exports are written to `./outputs/`.

In [ ]:
# Save final outputs
profile = profile_dataframe(df_raw)
(OUTPUT_DIR / "profile.csv").write_text("")
profile.to_csv(OUTPUT_DIR / "profile.csv", index=False)

df_enriched.to_parquet(OUTPUT_DIR / "enriched_companies.parquet", index=False)
df_enriched.to_csv(OUTPUT_DIR / "enriched_companies.csv", index=False)
print("Files written to:", OUTPUT_DIR.resolve())
list(OUTPUT_DIR.glob("*"))